In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import pandas as pd
import pickle

import json
import numpy as np

import re

In [3]:
from references_normalization import *

In [4]:
DATA_PATH="data/"

In [5]:
with open(DATA_PATH + "extracted_questions_20250207_final.pkl", "rb") as input_file:
    questions = pickle.load(input_file)

In [6]:
len(questions)

715

### Consolidate a list of unique legal document references

In [7]:
unique_references, questions_without_references = return_unique_references(questions)

{'question_number': '019', 'question_summary': 'RENDIMENTOS RECEBIDOS ACUMULADAMENTE (RRA)', 'question_text': 'O contribuinte que optar pelo desconto simplificado pode excluir os honorários advocatícios pagos referentes a rendimentos recebidos acumuladamente, por decisão judicial?', 'answer': ['Sim. O contribuinte, independentemente da opção pelo desconto simplificado ou não, pode informar como', 'rendimento tributável o valor recebido, excluindo os honorários pagos na proporção dos rendimentos', 'tributáveis.', 'Consulte, ainda, as perguntas 229, 248, 249, 251 e 439'], 'answer_cleaned': ['Sim. O contribuinte, independentemente da opção pelo desconto simplificado ou não, pode informar como', 'rendimento tributável o valor recebido, excluindo os honorários pagos na proporção dos rendimentos', 'tributáveis.', 'Consulte, ainda, as perguntas 229, 248, 249, 251 e 439'], 'references': [], 'linked_questions': [], 'formatted_references': None} ― Empty formatted_references field.

{'question_nu

In [8]:
len(questions_without_references)

146

In [16]:
sorted_unique_references = np.sort(list(unique_references.keys()))

In [21]:
sorted_unique_references.shape

(429,)

### Group the references by initial letter before sending to LLM for deduplication

In [22]:
from itertools import groupby

In [17]:
unique_references_by_initial_letter = {key: list(group) for key, group in groupby(sorted_unique_references, key=lambda x: x[0])}

In [19]:
unique_references_by_initial_letter.keys()

dict_keys(['A', 'C', 'D', 'I', 'L', 'M', 'N', 'P', 'R', 'S', 'e'])

In [23]:
group_counts = {}

for key, value in unique_references_by_initial_letter.items():
    group_counts[key] = len(value)

In [24]:
group_counts

{'A': 42,
 'C': 7,
 'D': 34,
 'I': 43,
 'L': 107,
 'M': 5,
 'N': 6,
 'P': 49,
 'R': 8,
 'S': 127,
 'e': 1}

### Deduplicate the references using LLM

In [27]:
from llm_access import *

import time

#### API keys
Expect to have a json file containing all the API keys needed. Groq key should be under "groq" identifier.

In [28]:
API_KEYS_FILE="/work/api_keys_20240427.json"

In [29]:
groq_key = json.load(open(API_KEYS_FILE))['groq']

#### Create the Groq model inteface to use

In [30]:
groq_llama33_70b_interface = groq_access(groq_key, GROQ_LLAMA3_3_70B_MODEL)

In [33]:
processing_start = time.time()

llama_deduplicated_reference_titles = []
total_prompt_tokens = 0
total_completion_tokens = 0

for initial_letter, references in unique_references_by_initial_letter.items():
    print("\n*****************************************")
    print(f"Processing titles starting with {initial_letter}\n")
    print("*****************************************\n")

    if len(references) > 1:    
        result = legal_refereces_titles_deduplication(groq_llama33_70b_interface, references)
    
        print(f">> Original titles count={len(references)}; deduplicated titles count={len(result['deduplicated-references'])}")
    
        llama_deduplicated_reference_titles += result["deduplicated-references"]
    
        total_prompt_tokens += result['prompt_tokens']
        total_completion_tokens += result['completion_tokens']
    else:
        print(f">> Single document directly added to the list")
        llama_deduplicated_reference_titles += references

processing_end = time.time()


*****************************************
Processing titles starting with A

*****************************************



Titles deduplication. 42 elements


Lista de documentos:
AD PGFN nº 3 de 18 de setembro de 2008
ADI RFB nº 12, de 2016
Acordo sobre o Novo Banco de Desenvolvimento
Anexo ao Decreto nº 361, de 10 de dezembro de 1991
Ato Declaratório Cosar nº 47, de 27 de novembro de 2000
Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019
Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024
Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005
Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016
Ato Declaratório Interpretativo RFB nº 18, de 6 de dezembro de 2007
Ato Declaratório Interpretativo RFB nº 3, de 27 de abril de 2016
Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004
Ato Declaratório Interpretativo SRF nº 2, de 27 de março de 2007
Ato Declaratório Interpretativo SRF nº 24, de 4 de outubro de 2004
Ato Declaratório Inter

In [34]:
print(f"Total time to process the legal references: {processing_end - processing_start}")

Total time to process the legal references: 41.84665870666504


In [204]:
print(f"total_prompt_tokens={total_prompt_tokens}, total_completion_tokens={total_completion_tokens}")

total_prompt_tokens=11398, total_completion_tokens=9232


In [35]:
len(llama_deduplicated_reference_titles)

404

In [37]:
with open(DATA_PATH + "llama_deduplicated_reference_titles_20250210.pkl", "wb") as output_file:
    pickle.dump(llama_deduplicated_reference_titles, output_file, pickle.HIGHEST_PROTOCOL)

### Check which were the removed references

In [55]:
np.setdiff1d(np.sort([normalize_document_title(which_title) for which_title in sorted_unique_references]), 
             np.sort([normalize_document_title(which_title) for which_title in llama_deduplicated_reference_titles]))

array(['ato declaratório interpretativo srf nº 8, de 2003',
       'ato declaratório normativo cosit nº 19, de 1998',
       'constituição federal, de 1988', 'decreto-lei nº 5.844, de 1943',
       'lei nº 7.713, de 1988', 'lei nº 8.383, de 1991',
       'lei nº 8.981, de 1995', 'lei nº 9.249, de 1995',
       'lei_9.250_26_fevereiro_1995', 'lei_9.718_23_novembro_1998',
       'lei_9250_26_dezembro_1995',
       'regulamento do imposto sobre a renda - rir/2018',
       'regulamento do imposto sobre a renda rir/2018, aprovado pelo decreto_9.580_22_novembro_2018',
       'regulamento do imposto sobre a renda – rir/2018',
       'regulamento do imposto sobre a renda – rir/2018, aprovado pelo decreto_9.580_22_novembro_2018'],
      dtype='<U111')

In [58]:
llama_deduplicated_reference_titles

['AD PGFN nº 3 de 18 de setembro de 2008',
 'ADI RFB nº 12, de 2016',
 'Acordo sobre o Novo Banco de Desenvolvimento',
 'Anexo ao Decreto nº 361, de 10 de dezembro de 1991',
 'Ato Declaratório Cosar nº 47, de 27 de novembro de 2000',
 'Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019',
 'Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024',
 'Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005',
 'Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016',
 'Ato Declaratório Interpretativo RFB nº 18, de 6 de dezembro de 2007',
 'Ato Declaratório Interpretativo RFB nº 3, de 27 de abril de 2016',
 'Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004',
 'Ato Declaratório Interpretativo SRF nº 24, de 4 de outubro de 2004',
 'Ato Declaratório Interpretativo SRF nº 26, de 26 de dezembro de 2003',
 'Ato Declaratório Interpretativo SRF nº 2, de 27 de março de 2007',
 'Ato Declaratório Interpretativo SRF nº 8, de 23 de abril de 2003',
 'At

### Manual fixes

Here manual fixes are added to scenarios where the automatic processing failed to acknowledge duplicates, or wrongly removed different documents.

#### Merge

"Acordo sobre o Novo Banco de Desenvolvimento" is the annex name in "Decreto nº 8.624, de 29 de dezembro de 2015", not a different document.

In [ ]:
llama_deduplicated_reference_titles.index('Decreto nº 8.624, de 29 de dezembro de 2015')

In [61]:
llama_deduplicated_reference_titles.index('Acordo sobre o Novo Banco de Desenvolvimento')

2

In [65]:
llama_deduplicated_reference_titles[62] = 'Decreto nº 8.624, de 29 de dezembro de 2015' + ", " + 'Acordo sobre o Novo Banco de Desenvolvimento'

In [66]:
llama_deduplicated_reference_titles[62]

'Decreto nº 8.624, de 29 de dezembro de 2015, Acordo sobre o Novo Banco de Desenvolvimento'

In [67]:
llama_deduplicated_reference_titles.pop(2)

'Acordo sobre o Novo Banco de Desenvolvimento'

In [68]:
len(llama_deduplicated_reference_titles)

403

#### Include missing

"Ato Declaratório Normativo Cosit nº 19, de 1998" has been wrongly removed by LLM.

In [69]:
[item for item in llama_deduplicated_reference_titles if "ato declaratório normativo cosit nº 19" in item.lower()]

['Ato Declaratório Normativo Cosit nº 19, de 25 de outubro de 2000']

In [70]:
llama_deduplicated_reference_titles.append("Ato Declaratório Normativo Cosit nº 19, de 1998")

In [71]:
len(llama_deduplicated_reference_titles)

404

#### Remove additional

"e-CAC" has been wrongly considered as a legal reference.

In [100]:
llama_deduplicated_reference_titles.index("e-CAC")

402

In [101]:
llama_deduplicated_reference_titles.pop(402)

'e-CAC'

#### Save final result

In [102]:
with open(DATA_PATH + "llama_deduplicated_reference_titles_20250210_fixed.pkl", "wb") as output_file:
    pickle.dump(llama_deduplicated_reference_titles, output_file, pickle.HIGHEST_PROTOCOL)

## Map extracted reference filenames to one of the deduplicated ones

In [85]:
doc_matcher = legalDocumentsMatcher(DATA_PATH + "llama_deduplicated_reference_titles_20250210_fixed.pkl")

In [9]:
with open(DATA_PATH + "normas_full.json", 'r') as file:
    normas_json = json.load(file)

In [10]:
all_extracted_docs_titles = np.sort(list(normas_json.keys()))

In [11]:
all_extracted_docs_titles.shape

(332,)

In [13]:
extracted_documents_parts = [split_document_name(which_document.split(".txt")[0]) for which_document in all_extracted_docs_titles]

In [87]:
extracted_documents_matches = []

for extracted_id, extracted_doc in enumerate(extracted_documents_parts):

    result = doc_matcher.get_best_match_for_parts(extracted_doc)

    extracted_documents_matches.append({"extracted_title": all_extracted_docs_titles[extracted_id],
                                        "title_f1": result["title_f1"],
                                        "number_f1": result["number_f1"],
                                        "date_f1": result["date_f1"],
                                        "best_matches": result["best_matches"],})

In [88]:
extracted_documents_matches

[{'extracted_title': 'Ato Declaratório Cosar nº 47.txt',
  'title_f1': 1.0,
  'number_f1': 1.0,
  'date_f1': 0,
  'best_matches': ['Ato Declaratório Cosar nº 47, de 27 de novembro de 2000']},
 {'extracted_title': 'Ato Declaratório Interpretativo RFB nº 1.txt',
  'title_f1': 1.0,
  'number_f1': 1.0,
  'date_f1': 0,
  'best_matches': ['Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016']},
 {'extracted_title': 'Ato Declaratório Interpretativo SRF nº 2.txt',
  'title_f1': 1.0,
  'number_f1': 1.0,
  'date_f1': 0,
  'best_matches': ['Ato Declaratório Interpretativo SRF nº 2, de 27 de março de 2007']},
 {'extracted_title': 'Ato Declaratório Interpretativo SRF nº 24.txt',
  'title_f1': 1.0,
  'number_f1': 1.0,
  'date_f1': 0,
  'best_matches': ['Ato Declaratório Interpretativo SRF nº 24, de 4 de outubro de 2004']},
 {'extracted_title': 'Ato Declaratório Interpretativo SRF nº 5.txt',
  'title_f1': 0.6666666666666665,
  'number_f1': 1.0,
  'date_f1': 0,
  'best_matches': ['Ato D

In [89]:
extracted_documents_parts[4]

['Ato Declaratório Interpretativo SRF ', 'nº 5', '5', '']

In [93]:
with open(DATA_PATH + "extracted_document_matches_20250210.pkl", "wb") as output_file:
    pickle.dump(extracted_documents_parts, output_file, pickle.HIGHEST_PROTOCOL)

#### Check the exact matches

In [91]:
exact_matches = []
multiple_matches = []

for doc_match in extracted_documents_matches:
    if len(doc_match['best_matches']) == 1:
        exact_matches.append(doc_match)
    else:
        multiple_matches.append(doc_match)

In [92]:
print(f"Number of exact matches={len(exact_matches)}; number of multiple matches={len(multiple_matches)}")

Number of exact matches=266; number of multiple matches=66


In [56]:
import pandas as pd

In [57]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 500)

In [58]:
pd.DataFrame(multiple_matches)

,extracted_title,title_f1,number_f1,date_f1,best_matches
0,Ato Declaratório PGFN nº 13.txt,1.000000,0.0,0,"[Ato Declaratório PGFN nº 5, de 3 de maio de 2016, Ato Declaratório PGFN nº 1, de 12 de março de 2018, Ato Declaratório PGFN nº 3, de 30 de março de 2016, Ato Declaratório PGFN nº 4, de 16 de novembro de 2006, Ato Declaratório PGFN nº 9, de 20 de dezembro de 2011]"
1,Ato Declaratório PGFN nº 2.txt,0.666667,1.0,0,"[Ato Declaratório Interpretativo SRF nº 2, de 27 de março de 2007, Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024]"
2,Decreto nº 21.177.txt,1.000000,0.0,0,"[Decreto nº 95.711, de 10 de fevereiro de 1988, Decreto nº 61.078, de 26 de julho de 1967, Decreto nº 93.153, de 22 de agosto de 1986, Convênio Constitutivo da Corporação Interamericana de Investimentos, Decreto nº 9.580, de 22 de novembro de 2018, Decreto nº 87.563, de 13 de setembro de 1982, Decreto nº 86.365, de 15 de setembro de 1981, Decreto nº 86.084, de 8 de junho de 1981, Decreto nº 85.306, de 30 de outubro de 1980, capítulo II, art. 7, item 1, “e”, do Protocolo sobre Privilégios, Isenções e Imunidades da Intelsat, Decreto nº 8.624, de 29 de dezembro de 2015, Acordo sobre o Novo Banco de Desenvolvimento, Decreto nº 8.289, de 25 de julho de 2014, Decreto nº 71.733, de 18 de janeiro de 1973, Decreto nº 70.951, de 9 de agosto de 1972, Decreto nº 67.542, de 12 de novembro de 1970, Decreto nº 63.151, de 22 de agosto de 1968, Decreto nº 85.801, de 10 de março de 1981, Decreto nº 59.566, de 14 de novembro de 1966, Decreto nº 57.942, de 10 de março de 1966, Decreto nº 57.784, de 11 de fevereiro de 1966, Decreto nº 56.435, de 8 de junho de 1965 - Convenção de Viena sobre Relações Diplomáticas, Decreto nº 52.288, de 24 de julho de 1963, Decreto nº 50.656, de 24 de maio de 1961, Decreto nº 5.128, de 6 de julho de 2004, Decreto nº 27.784, de 16 de fevereiro de 1950, Decreto nº 59.308, de 23 de setembro de 1966]"
3,Decreto nº 3.000.txt,1.000000,0.0,0,"[Decreto nº 95.711, de 10 de fevereiro de 1988, Decreto nº 61.078, de 26 de julho de 1967, Decreto nº 93.153, de 22 de agosto de 1986, Convênio Constitutivo da Corporação Interamericana de Investimentos, Decreto nº 9.580, de 22 de novembro de 2018, Decreto nº 87.563, de 13 de setembro de 1982, Decreto nº 86.365, de 15 de setembro de 1981, Decreto nº 86.084, de 8 de junho de 1981, Decreto nº 85.306, de 30 de outubro de 1980, capítulo II, art. 7, item 1, “e”, do Protocolo sobre Privilégios, Isenções e Imunidades da Intelsat, Decreto nº 8.624, de 29 de dezembro de 2015, Acordo sobre o Novo Banco de Desenvolvimento, Decreto nº 8.289, de 25 de julho de 2014, Decreto nº 71.733, de 18 de janeiro de 1973, Decreto nº 70.951, de 9 de agosto de 1972, Decreto nº 67.542, de 12 de novembro de 1970, Decreto nº 63.151, de 22 de agosto de 1968, Decreto nº 85.801, de 10 de março de 1981, Decreto nº 59.566, de 14 de novembro de 1966, Decreto nº 57.942, de 10 de março de 1966, Decreto nº 57.784, de 11 de fevereiro de 1966, Decreto nº 56.435, de 8 de junho de 1965 - Convenção de Viena sobre Relações Diplomáticas, Decreto nº 52.288, de 24 de julho de 1963, Decreto nº 50.656, de 24 de maio de 1961, Decreto nº 5.128, de 6 de julho de 2004, Decreto nº 27.784, de 16 de fevereiro de 1950, Decreto nº 59.308, de 23 de setembro de 1966]"
4,Decreto nº 4.897.txt,1.000000,0.0,0,"[Decreto nº 95.711, de 10 de fevereiro de 1988, Decreto nº 61.078, de 26 de julho de 1967, Decreto nº 93.153, de 22 de agosto de 1986, Convênio Constitutivo da Corporação Interamericana de Investimentos, Decreto nº 9.580, de 22 de novembro de 2018, Decreto nº 87.563, de 13 de setembro de 1982, Decreto nº 86.365, de 15 de setembro de 1981, Decreto nº 86.084, de 8 de junho de 1981, Decreto nº 85.306, de 30 de outubro de 1980, capítulo II, art. 7, item 1, “e”, do Protocolo sobre Privilégios, Isenções e Imunidades da Intelsat, Decreto nº 8.624, de 29 de dezembro de 2015, Acordo sobre o Novo Banco de Desenvolvimento, Decreto nº 8.289, de 25 de julho de 2014, Decreto nº

In [59]:
multiple_matches[22]

{'extracted_title': 'Lei nº 10.865.txt',
 'title_f1': 1.0,
 'number_f1': 0,
 'date_f1': 0,
 'best_matches': ['Lei nº 9.394, de 20 de dezembro de 1996',
  'Lei nº 12.350, de 20 de dezembro de 2010',
  'Lei nº 7.713, de 22 de dezembro de 1998',
  'Lei nº 7.713, de 22 de dezembro de 1988',
  'Lei nº 7.290, de 19 de dezembro de 1984',
  'Lei nº 7.070, de 20 de dezembro de 1982',
  'Lei nº 6.766, de 19 de dezembro de 1979',
  'Lei nº 6.404, de 15 de dezembro de 1976',
  'Lei nº 6.015, de 31 de dezembro de 1973',
  'Lei nº 5.809, de 10 de outubro de 1972',
  'Lei nº 5.172, de 25 de outubro de 1966 - Código Tributário Nacional – CTN',
  'Lei nº 4.862, de 29 de novembro de 1965',
  'Lei nº 4.591, de 16 de dezembro de 1964',
  'Lei nº 4.506, de 30 de novembro de 1964',
  'Lei nº 4.504, de 30 de novembro de 1964',
  'Lei nº 14.754, de 12 de dezembro de 2023',
  'Lei nº 14.286, de 29 de dezembro de 2021',
  'Lei nº 14.260, de 8 de dezembro de 2021',
  'Lei nº 14.119, de 13 de janeiro de 2021',
  

In [60]:
pd.DataFrame(exact_matches)

,extracted_title,title_f1,number_f1,date_f1,best_matches
0,Ato Declaratório Cosar nº 47.txt,1.000000,1.0,0.000000,"[Ato Declaratório Cosar nº 47, de 27 de novembro de 2000]"
1,Ato Declaratório Interpretativo RFB nº 1.txt,1.000000,1.0,0.000000,"[Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016]"
2,Ato Declaratório Interpretativo SRF nº 2.txt,1.000000,1.0,0.000000,"[Ato Declaratório Interpretativo SRF nº 2, de 27 de março de 2007]"
3,Ato Declaratório Interpretativo SRF nº 24.txt,1.000000,1.0,0.000000,"[Ato Declaratório Interpretativo SRF nº 24, de 4 de outubro de 2004]"
4,Ato Declaratório Interpretativo SRF nº 5.txt,0.666667,1.0,0.000000,"[Ato Declaratório PGFN nº 5, de 3 de maio de 2016]"
5,Ato Declaratório PGFN nº 1.txt,1.000000,1.0,0.000000,"[Ato Declaratório PGFN nº 1, de 12 de março de 2018]"
6,Ato Declaratório PGFN nº 14.txt,0.666667,1.0,0.000000,"[Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004]"
7,Ato Declaratório PGFN nº 3.txt,1.000000,1.0,0.000000,"[Ato Declaratório PGFN nº 3, de 30 de março de 2016]"
8,Ato Declaratório PGFN nº 4.txt,1.000000,1.0,0.000000,"[Ato Declaratório PGFN nº 4, de 16 de novembro de 2006]"
9,Ato Declaratório PGFN nº 5.txt,1.000000,1.0,0.000000,"[Ato Declaratório PGFN nº 5, de 3 de maio de 2016]"
